# Gestión de Hospedaje

Este notebook administra las necesidades de hospedaje para los 21 invitados que lo requieren.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Cargar Datos

In [ ]:
df_hospedaje = pd.read_csv('../datos/hospedaje.csv')
df_invitados = pd.read_csv('../datos/invitados.csv')

total_personas_hospedaje = df_hospedaje['personas_hospedaje'].sum()
print(f"Total de personas que requieren hospedaje: {total_personas_hospedaje}")
print(f"Total de grupos: {len(df_hospedaje)}")

## 2. Estado Actual del Hospedaje

In [ ]:
# Calcular estadísticas
asignados = df_hospedaje[df_hospedaje['asignado'] == True]
pendientes = df_hospedaje[df_hospedaje['asignado'] == False]

personas_asignadas = asignados['personas_hospedaje'].sum()
personas_pendientes = pendientes['personas_hospedaje'].sum()

costo_total_estimado = df_hospedaje['costo_estimado'].sum()
costo_asignados = asignados['costo_estimado'].sum()

print("\n" + "="*60)
print("           ESTADO DEL HOSPEDAJE")
print("="*60 + "\n")

print(f"🏨 GENERAL:")
print(f"   Total personas:             {total_personas_hospedaje}")
print(f"   Total grupos:               {len(df_hospedaje)}")
print(f"   Costo total estimado:       ${costo_total_estimado:,.0f}")

print(f"\n✅ ASIGNADOS:")
print(f"   Grupos asignados:           {len(asignados)} ({len(asignados)/len(df_hospedaje)*100:.1f}%)")
print(f"   Personas asignadas:         {personas_asignadas} ({personas_asignadas/total_personas_hospedaje*100:.1f}%)")
print(f"   Costo asignados:            ${costo_asignados:,.0f}")

print(f"\n⏳ PENDIENTES:")
print(f"   Grupos pendientes:          {len(pendientes)} ({len(pendientes)/len(df_hospedaje)*100:.1f}%)")
print(f"   Personas pendientes:        {personas_pendientes} ({personas_pendientes/total_personas_hospedaje*100:.1f}%)")
print(f"   Costo estimado pendiente:   ${(costo_total_estimado - costo_asignados):,.0f}")

print("\n" + "="*60)

## 3. Lista Detallada de Hospedaje

In [ ]:
print("\n" + "="*80)
print("                    LISTA DE HOSPEDAJE")
print("="*80 + "\n")

print(f"{'Estado':<8} {'Nombre':<25} {'Personas':<10} {'Habitación':<15} {'Costo':<15}")
print("-" * 80)

for _, row in df_hospedaje.iterrows():
    estado = '✅' if row['asignado'] else '⏳'
    habitacion = row['tipo_habitacion'] if pd.notna(row['tipo_habitacion']) else 'Sin asignar'
    costo = f"${row['costo_estimado']:,.0f}" if row['costo_estimado'] > 0 else '-'
    
    print(f"{estado:<8} {row['nombre_invitado']:<25} {row['personas_hospedaje']:<10} {habitacion:<15} {costo:<15}")
    
    if pd.notna(row['notas']) and row['notas']:
        print(f"         📝 {row['notas']}")

print("\n" + "="*80)
print(f"{'TOTAL':<8} {'':<25} {total_personas_hospedaje:<10} {'':<15} ${costo_total_estimado:,.0f}")
print("="*80)

## 4. Cálculo de Habitaciones Necesarias

In [ ]:
print("\n🛏️  ANÁLISIS DE HABITACIONES NECESARIAS\n")
print("="*60)

# Contar por número de personas
distribucion = df_hospedaje['personas_hospedaje'].value_counts().sort_index()

print("\nDistribución de grupos por tamaño:")
for personas, cantidad in distribucion.items():
    print(f"   {cantidad} grupo(s) de {personas} persona(s)")

# Sugerencias de habitaciones
grupos_1 = len(df_hospedaje[df_hospedaje['personas_hospedaje'] == 1])
grupos_2 = len(df_hospedaje[df_hospedaje['personas_hospedaje'] == 2])
grupos_3 = len(df_hospedaje[df_hospedaje['personas_hospedaje'] == 3])
grupos_4_o_mas = len(df_hospedaje[df_hospedaje['personas_hospedaje'] >= 4])

print("\n💡 SUGERENCIAS DE HABITACIONES:\n")

# Habitaciones individuales o dobles para personas solas
if grupos_1 > 0:
    print(f"   {grupos_1} habitación(es) individual(es) o doble(s) para 1 persona")

# Habitaciones dobles para parejas
if grupos_2 > 0:
    print(f"   {grupos_2} habitación(es) doble(s) para 2 personas")

# Habitaciones triples o suite
if grupos_3 > 0:
    print(f"   {grupos_3} habitación(es) triple(s) o suite(s) para 3 personas")

if grupos_4_o_mas > 0:
    print(f"   {grupos_4_o_mas} suite(s) o habitación(es) familiar(es) para 4+ personas")

# Estimación de habitaciones
habitaciones_minimas = grupos_1 + grupos_2 + grupos_3 + grupos_4_o_mas
print(f"\n📊 Mínimo de habitaciones necesarias: {habitaciones_minimas}")
print(f"   (Considerando cada grupo en su propia habitación)")

# Alternativa: compartir habitaciones
if grupos_1 >= 2:
    habitaciones_compartidas = habitaciones_minimas - (grupos_1 // 2)
    print(f"\n💡 ALTERNATIVA: {habitaciones_compartidas} habitaciones")
    print(f"   (Si personas solas comparten habitaciones dobles)")

print("\n" + "="*60)

## 5. Visualizaciones

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Estado de asignación
estado_data = [personas_asignadas, personas_pendientes]
estado_labels = [f'Asignadas\n({personas_asignadas})', f'Pendientes\n({personas_pendientes})']
colors_estado = ['#2ecc71', '#f39c12']
axes[0, 0].pie(estado_data, labels=estado_labels, autopct='%1.1f%%',
               startangle=90, colors=colors_estado)
axes[0, 0].set_title('Estado de Asignación de Hospedaje', fontsize=14, fontweight='bold')

# 2. Distribución por tamaño de grupo
distribucion_plot = df_hospedaje['personas_hospedaje'].value_counts().sort_index()
axes[0, 1].bar(distribucion_plot.index, distribucion_plot.values, color='#3498db')
axes[0, 1].set_xlabel('Personas por Grupo', fontsize=12)
axes[0, 1].set_ylabel('Número de Grupos', fontsize=12)
axes[0, 1].set_title('Distribución de Grupos por Tamaño', fontsize=14, fontweight='bold')
for i, v in zip(distribucion_plot.index, distribucion_plot.values):
    axes[0, 1].text(i, v, str(v), ha='center', va='bottom', fontsize=11, fontweight='bold')

# 3. Costos de hospedaje
if costo_total_estimado > 0:
    # Costos por grupo (solo los que tienen costo)
    df_con_costo = df_hospedaje[df_hospedaje['costo_estimado'] > 0].sort_values('costo_estimado', ascending=True)
    if len(df_con_costo) > 0:
        axes[1, 0].barh(df_con_costo['nombre_invitado'], df_con_costo['costo_estimado'], color='#e74c3c')
        axes[1, 0].set_xlabel('Costo Estimado (COP)', fontsize=12)
        axes[1, 0].set_title('Costo de Hospedaje por Grupo', fontsize=14, fontweight='bold')
    else:
        axes[1, 0].text(0.5, 0.5, 'Sin costos registrados', ha='center', va='center', fontsize=12)
        axes[1, 0].set_xlim(0, 1)
        axes[1, 0].set_ylim(0, 1)
else:
    axes[1, 0].text(0.5, 0.5, 'Sin costos registrados', ha='center', va='center', fontsize=12)
    axes[1, 0].set_xlim(0, 1)
    axes[1, 0].set_ylim(0, 1)

# 4. Progreso de asignación
categorias_prog = ['Asignados', 'Pendientes']
valores_prog = [len(asignados), len(pendientes)]
colors_prog = ['#2ecc71', '#f39c12']
axes[1, 1].bar(categorias_prog, valores_prog, color=colors_prog)
axes[1, 1].set_ylabel('Número de Grupos', fontsize=12)
axes[1, 1].set_title('Progreso de Asignación de Habitaciones', fontsize=14, fontweight='bold')
for i, v in enumerate(valores_prog):
    axes[1, 1].text(i, v, str(v), ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Hospedajes Pendientes de Asignar

In [ ]:
if len(pendientes) > 0:
    print("\n⏳ HOSPEDAJES PENDIENTES DE ASIGNAR\n")
    print("="*70)
    
    for _, row in pendientes.iterrows():
        print(f"\n👤 {row['nombre_invitado']}")
        print(f"   Personas: {row['personas_hospedaje']}")
        
        # Sugerencia de tipo de habitación
        if row['personas_hospedaje'] == 1:
            sugerencia = "Individual o Doble"
        elif row['personas_hospedaje'] == 2:
            sugerencia = "Doble"
        elif row['personas_hospedaje'] == 3:
            sugerencia = "Triple o Suite"
        else:
            sugerencia = "Suite o Familiar"
        
        print(f"   Sugerencia: {sugerencia}")
        
        if pd.notna(row['notas']) and row['notas']:
            print(f"   Notas: {row['notas']}")
    
    print("\n" + "="*70)
    print(f"\n📋 Total pendiente: {len(pendientes)} grupos, {personas_pendientes} personas")
else:
    print("\n✅ ¡Excelente! Todos los hospedajes han sido asignados.")

## 7. Estimación de Costos

In [ ]:
print("\n💰 ESTIMACIÓN DE COSTOS DE HOSPEDAJE\n")
print("="*60)

if costo_total_estimado > 0:
    print(f"\nCosto total estimado:       ${costo_total_estimado:,.0f}")
    print(f"Ya asignado:                ${costo_asignados:,.0f}")
    print(f"Pendiente por asignar:      ${(costo_total_estimado - costo_asignados):,.0f}")
    
    # Costo promedio por persona
    costo_por_persona = costo_total_estimado / total_personas_hospedaje
    print(f"\nCosto promedio por persona: ${costo_por_persona:,.0f}")
    
    # Proyección para pendientes si no tienen costo
    if personas_pendientes > 0 and costo_asignados == costo_total_estimado:
        print(f"\n⚠️  Hay {personas_pendientes} personas sin costo estimado")
else:
    print("\n⚠️  No hay costos registrados aún.")
    print("\n💡 GUÍA DE PRECIOS ESTIMADOS (por noche):")
    print("   - Habitación Individual:  $80,000 - $150,000")
    print("   - Habitación Doble:       $100,000 - $200,000")
    print("   - Habitación Triple:      $150,000 - $250,000")
    print("   - Suite Familiar:         $200,000 - $400,000")
    print("\n   Actualiza datos/hospedaje.csv con los costos reales.")

print("\n" + "="*60)

## 8. Checklist de Hospedaje

In [ ]:
print("\n✅ CHECKLIST DE HOSPEDAJE\n")
print("="*60)

# Verificar tareas
print("\n📋 TAREAS:")

# 1. Cotizaciones
if costo_total_estimado > 0:
    print("   ✅ Cotizaciones obtenidas")
else:
    print("   ⏳ Obtener cotizaciones de hoteles")

# 2. Asignaciones
if len(pendientes) == 0:
    print("   ✅ Todas las habitaciones asignadas")
else:
    print(f"   ⏳ Asignar habitaciones ({len(pendientes)} pendientes)")

# 3. Confirmaciones
confirmados_hospedaje = df_invitados[(df_invitados['hospedaje'] > 0) & (df_invitados['confirmado'] == True)]
if len(confirmados_hospedaje) == len(df_hospedaje):
    print("   ✅ Todos los invitados con hospedaje confirmaron")
else:
    pendientes_confirmar = len(df_hospedaje) - len(confirmados_hospedaje)
    print(f"   ⏳ Confirmar asistencia ({pendientes_confirmar} grupos pendientes)")

# 4. Reservas
print("   ⏳ Realizar reservas en hoteles")

# 5. Comunicación
print("   ⏳ Enviar información de hospedaje a invitados")

# 6. Transporte
print("   ⏳ Coordinar transporte hotel-evento-hotel")

print("\n💡 RECORDATORIOS:")
print("   - Reservar con al menos 2 meses de anticipación")
print("   - Negociar tarifa grupal si es posible")
print("   - Confirmar políticas de cancelación")
print("   - Verificar qué incluye cada habitación (desayuno, wifi, etc.)")
print("   - Considerar proximidad al lugar del evento")

print("\n" + "="*60)

## 9. Exportar Reporte de Hospedaje

In [ ]:
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'../datos/reporte_hospedaje_{timestamp}.xlsx'

with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    # Hoja 1: Lista completa
    df_hospedaje.to_excel(writer, sheet_name='Lista_Completa', index=False)
    
    # Hoja 2: Pendientes
    if len(pendientes) > 0:
        pendientes.to_excel(writer, sheet_name='Pendientes', index=False)
    
    # Hoja 3: Asignados
    if len(asignados) > 0:
        asignados.to_excel(writer, sheet_name='Asignados', index=False)
    
    # Hoja 4: Resumen
    resumen = pd.DataFrame({
        'Concepto': ['Total Personas', 'Total Grupos', 'Grupos Asignados', 'Grupos Pendientes', 
                     'Costo Total', 'Costo Asignados', 'Habitaciones Mínimas'],
        'Valor': [total_personas_hospedaje, len(df_hospedaje), len(asignados), len(pendientes),
                  f'${costo_total_estimado:,.0f}', f'${costo_asignados:,.0f}', habitaciones_minimas]
    })
    resumen.to_excel(writer, sheet_name='Resumen', index=False)

print(f"\n✅ Reporte de hospedaje exportado: {filename}")